In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = "--master local[*] --packages org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.0,org.apache.spark:spark-streaming-kafka-0-10_2.11:2.2.0 pyspark-shell"

In [2]:
import json
from pyspark.sql.functions import col, udf, explode
from pyspark.sql.types import *
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

sc = SparkContext('local')
spark = SparkSession(sc)

In [3]:
kafka_brokers = 'kafka:9092'
topic = 'sv-uploads-default-topic'

input_df = spark \
     .readStream \
     .format('kafka') \
     .option('kafka.bootstrap.servers', kafka_brokers) \
     .option('subscribe', topic) \
     .option("startingOffsets", "earliest") \
     .option("maxOffsetsPerTrigger", "10") \
     .load()

In [7]:
input_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [8]:
hadoop_conf = sc._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
hadoop_conf.set("fs.s3a.endpoint", "localstack:4572")
hadoop_conf.set("fs.s3a.access.key", "dummy")
hadoop_conf.set("fs.s3a.secret.key", "dummy")